# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

### Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

#### Import the mnist dataset from keras datasets

In [2]:
import keras
from keras.datasets import mnist

In [20]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [21]:
x_train.shape

(60000, 28, 28)

#### Creating two datasets one with digits below 5 and one with 5 and above

In [22]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
x_test_gt5 = x_test[y_test >= 5]         # np_utils.to_categorical
y_test_gt5 = y_test[y_test >= 5] - 5

### Check 

Verify shapes of x_train, y_train, x_test and y_test for both the datasets with the below given shapes.

In [23]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(30596, 28, 28)
(30596,)
(5139, 28, 28)
(5139,)


In [24]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


### Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST
### Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [25]:
x_train_lt5_reshape=x_train_lt5.reshape(x_train_lt5.shape[0],28,28,1)
x_train_lt5_reshape.shape

(30596, 28, 28, 1)

In [26]:
x_test_lt5_reshape=x_test_lt5.reshape(x_test_lt5.shape[0],28,28,1)
x_test_lt5_reshape.shape

(5139, 28, 28, 1)

### Change into float32 datatype and Normalize x_train and x_test by dividing it by 255.0

In [27]:
x_train_lt5_normalized=x_train_lt5_reshape.astype(float)/255

In [28]:
x_test_lt5_normalized=x_test_lt5_reshape.astype(float)/255

### Check

Verify the shapes of the X_train and X_test with the shapes given below.

In [29]:
print('X_train shape:', x_train_lt5_normalized.shape)
print('X_test shape:', x_test_lt5_normalized.shape)

X_train shape: (30596, 28, 28, 1)
X_test shape: (5139, 28, 28, 1)


### Use One-hot encoding to divide y_train and y_test into required no of output classes

In [31]:
y_train_lt5=keras.utils.to_categorical(y_train_lt5)

In [32]:
y_train_lt5.shape

(30596, 5)

In [37]:
y_train_lt5

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [33]:
y_test_lt5=keras.utils.to_categorical(y_test_lt5)

In [41]:
y_test_lt5.shape

(5139, 5)

### Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [35]:
from keras.models import Sequential

In [53]:
from keras.layers import Dense,Convolution2D,Activation,MaxPool2D,Dropout,Flatten

In [39]:
model=Sequential()

In [44]:
model.add(Convolution2D(32, 3, 3, input_shape=(28, 28, 1)))
model.add(Activation('relu'))

C:\Users\Shreyaspatil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(28, 28, 1...)`
  """Entry point for launching an IPython kernel.


In [46]:
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))

C:\Users\Shreyaspatil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  """Entry point for launching an IPython kernel.


In [50]:
#Max Pooling
model.add(MaxPool2D(pool_size=(2,2)))

In [51]:
# Dropout
model.add(Dropout(0.25))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [54]:
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

In [55]:
#Output
model.add(Dense(5))
model.add(Activation('softmax'))

In [56]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

### Print the training and test accuracy for 5 epochs

In [70]:
BATCH_SIZE = 32
EPOCHS = 5

In [123]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]

model.fit(x_train_lt5_normalized, y_train_lt5, batch_size=BATCH_SIZE, nb_epoch=EPOCHS, 
              validation_data=(x_test_lt5_normalized, y_test_lt5), callbacks=callback_list)

Train on 30596 samples, validate on 5139 samples
Epoch 1/5
   32/30596 [..............................] - ETA: 1:54 - loss: 0.0104 - acc: 1.0000

C:\Users\Shreyaspatil\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """
C:\Users\Shreyaspatil\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


30596/30596 [==============================] - 105s 3ms/step - loss: 0.0028 - acc: 0.9990 - val_loss: 0.0049 - val_acc: 0.9984
Epoch 2/5
30596/30596 [==============================] - 105s 3ms/step - loss: 0.0033 - acc: 0.9990 - val_loss: 0.0058 - val_acc: 0.9977
Epoch 3/5
30596/30596 [==============================] - 106s 3ms/step - loss: 0.0041 - acc: 0.9987 - val_loss: 0.0068 - val_acc: 0.9973
Epoch 4/5
30596/30596 [==============================] - 106s 3ms/step - loss: 0.0021 - acc: 0.9992 - val_loss: 0.0049 - val_acc: 0.9986
Epoch 5/5
30596/30596 [==============================] - 105s 3ms/step - loss: 0.0016 - acc: 0.9995 - val_loss: 0.0033 - val_acc: 0.9988


### Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

### Make only the dense layers to be trainable and convolutional layers to be non-trainable

#### Check model summary to see model layer names

In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 32)          0         
__________

In [66]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv2d_2
conv2d_2is not trainable

activation_1
activation_1is not trainable

conv2d_3
conv2d_3is not trainable

activation_2
activation_2is not trainable

max_pooling2d_1
max_pooling2d_1is not trainable

max_pooling2d_2
max_pooling2d_2is not trainable

dropout_1
dropout_1is not trainable

flatten_1
flatten_1is not trainable

dense_1
dense_1 is trainable

activation_3
activation_3is not trainable

dense_2
dense_2 is trainable

activation_4
activation_4is not trainable



### Do the required preprocessing for `x_train_gt5` also same as `x_train_lt5` and for `y_train_gt5` same as `y_train_lt5`

1. Reshape
2. Change to float32 datatype
3. Normalize (dividing with 255)
4. y_train and y_test Convert into one-hot vectors

In [61]:
x_train_gt5_reshape=x_train_gt5.reshape(x_train_gt5.shape[0],28,28,1)
x_train_gt5_reshape.shape

(29404, 28, 28, 1)

In [63]:
x_test_gt5_reshape=x_test_gt5.reshape(x_test_gt5.shape[0],28,28,1)
x_test_gt5_reshape.shape

(4861, 28, 28, 1)

In [64]:
x_train_gt5_normalized=x_train_gt5_reshape.astype(float)/255

In [65]:
x_test_gt5_normalized=x_test_gt5_reshape.astype(float)/255

In [67]:
y_train_gt5=keras.utils.to_categorical(y_train_gt5)

In [68]:
y_test_gt5=keras.utils.to_categorical(y_test_gt5)

### Check

Verify the shapes with the given below.

In [69]:
print(x_train_gt5_normalized.shape)
print(y_train_gt5.shape)
print(x_test_gt5_normalized.shape)
print(y_test_gt5.shape)

(29404, 28, 28, 1)
(29404, 5)
(4861, 28, 28, 1)
(4861, 5)


## Print the accuracy for classification of digits 5 to 9

In [77]:
model.fit(x_train_gt5_normalized, y_train_gt5, batch_size=BATCH_SIZE, nb_epoch=EPOCHS, 
              validation_data=(x_test_gt5_normalized, y_test_gt5), callbacks=callback_list)

Train on 29404 samples, validate on 4861 samples
Epoch 1/5
   32/29404 [..............................] - ETA: 1:46 - loss: 0.0024 - acc: 1.0000

C:\Users\Shreyaspatil\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  
C:\Users\Shreyaspatil\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


29404/29404 [==============================] - 100s 3ms/step - loss: 0.0107 - acc: 0.9961 - val_loss: 0.0191 - val_acc: 0.9961
Epoch 2/5
29404/29404 [==============================] - 98s 3ms/step - loss: 0.0082 - acc: 0.9970 - val_loss: 0.0209 - val_acc: 0.9959
Epoch 3/5
29404/29404 [==============================] - 101s 3ms/step - loss: 0.0089 - acc: 0.9969 - val_loss: 0.0200 - val_acc: 0.9957
Epoch 4/5
29404/29404 [==============================] - 100s 3ms/step - loss: 0.0067 - acc: 0.9976 - val_loss: 0.0192 - val_acc: 0.9953
Epoch 5/5
29404/29404 [==============================] - 103s 3ms/step - loss: 0.0053 - acc: 0.9982 - val_loss: 0.0238 - val_acc: 0.9947


# Text classification using TF-IDF

###  Load the dataset from sklearn.datasets

In [78]:
from sklearn.datasets import fetch_20newsgroups

In [79]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### Training data

In [81]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### Test data

In [82]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [83]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2], dtype=int64)

In [84]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [85]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [102]:
# TfidfVectorizer
vect = TfidfVectorizer()
twenty_train_tfidf=vect.fit_transform(twenty_train.data)

In [103]:
twenty_train_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [104]:
twenty_test_tfidf=vect.transform(twenty_test.data)

### Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [94]:
from sklearn.linear_model import LogisticRegression

In [117]:
model_lg=LogisticRegression(multi_class='ovr')

In [118]:
model_lg.fit(twenty_train_tfidf,twenty_train.target)

C:\Users\Shreyaspatil\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [119]:
y_pred=model_lg.predict(twenty_test_tfidf)

In [120]:
from sklearn.metrics import accuracy_score

In [121]:
accuracy_score(twenty_test.target,y_pred)

0.8868175765645806